# autoMLST Wrapper
Summary of [autoMLST Wrapper](https://github.com/KatSteinke/automlst-simplified-wrapper) results from project: `[{{ project().name }}]`

## Description
This report provides an overview of the result from [autoMLST Wrapper](https://github.com/KatSteinke/automlst-simplified-wrapper), a modified version of [autoMLST](https://bitbucket.org/ziemertlab/automlst) tailored for simplified usability. By integrating a straightforward wrapper script, this fork eliminates the need for additional organism selection steps, streamlining the process for users.

In [ ]:
import pandas as pd
from pathlib import Path
from IPython.display import display, Markdown, HTML
import json
import seaborn as sns
import matplotlib.colors as mcolors

import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', message='.*method overwritten by.*')

from itables import to_html_datatable as DT
import itables.options as opt
opt.css = """
.itables table td { font-style: italic; font-size: .8em;}
.itables table th { font-style: oblique; font-size: .8em; }
"""
opt.classes = ["display", "compact"]
opt.lengthMenu = [5, 10, 20, 50, 100, 200, 500]

report_dir = Path("../")

%load_ext rpy2.ipython

In [ ]:
report_dir = Path("../")

dependency_version = report_dir / "metadata/dependency_versions.json"
with open(dependency_version, "r") as file:
    dependency_version = json.load(file)
antismash_version = dependency_version["antismash"]

In [ ]:
with open(report_dir / "automlst_wrapper/final.newick", "r") as f:
    data = f.readlines()

value_to_replace = [i.split(":")[0] for i in data[0].replace("(", "").split(",")]

new_dict = {}
df = pd.read_csv("../automlst_wrapper/df_genomes_tree.csv")
genome_ids = list(df.genome_id)
for g in genome_ids:
    for v in value_to_replace:
        if v.startswith(g.split(".")[0]):
            new_dict[v] = g
            value_to_replace.remove(v)

data = data[0]
for k in new_dict.keys():
    data = data.replace(k, new_dict[k])

outfile = Path("assets/data/final_corrected.newick")
outfile.parent.mkdir(parents=True, exist_ok=True)
with open(outfile, "w") as f:
    f.write(data)

## Visualization
The tree visualization represents the phylogenetic relationships between various strains of the genus. This visualization aids in understanding the genetic diversity and evolutionary history of these genomes.

In [ ]:
%%capture
%%R
suppressPackageStartupMessages({
  library("treeio")
  library("ggtree")
  library("tidyverse")
  library("ggstar")
  library("ggnewscale")
  library("ggtreeExtra")
  library("phangorn")
  library("RColorBrewer")
})

In [ ]:
%%R  -w 800 -h 800
tree <- read.tree("assets/data/final_corrected.newick")
#data <- read.csv("../automlst_wrapper/df_genomes_tree.csv")
data <- read.csv("../tables/df_gtdb_meta.csv")

# Shorten the Organism column
data$Organism_short <- sub("^s__([A-Za-z])[a-z]*.*\\s", "\\1. ", data$Organism) # Shorten the genus name
data$Organism_short <- sub("^s__", "", data$Organism_short)  # Remove 's__'

# midpoint root
tree <- phangorn::midpoint(tree)
tree <- ladderize(reorder(tree))

# Add labels to all nodes

tree$node.label <- paste0("N", 1:(Nnode(tree) + Ntip(tree)))

# Write the tree with internal node IDs to a new Newick file
if (!dir.exists("assets/iTOL_annotation")) {
  dir.create("assets/iTOL_annotation", recursive = TRUE)
}

write.tree(tree, file = "assets/iTOL_annotation/automlst_tree_with_ids.newick")

# Get the unique genera
genera <- unique(data$Genus)

# Initialize the plot
p <- ggtree(tree, 
            #layout="fan", 
            size=1, open.angle=5, branch.length='none')

# Create a vector of colors
colors <- brewer.pal(length(genera), "Set1")

# Create a mapping from genera to colors
genus_to_color <- setNames(colors, genera)

# Initialize new columns for color annotation
data$tree_color <- NA
data$tree_color_label <- NA
data$tree_color_MRCA <- NA

# Add a clade label for each genus
for (genus in genera) {
  # Get the tips that belong to this genus
  genus_tips <- data$genome_id[data$Genus == genus]
  
  # Find the MRCA of these tips
  mrca_node <- getMRCA(tree, genus_tips)
  
  # Subtract the number of tips from the MRCA node index
  internal_node_index <- mrca_node - Ntip(tree)

  # Get the label of the internal node
  mrca_label <- tree$node.label[internal_node_index]

  # Add the color, label, and MRCA to the new columns
  data$tree_color[data$Genus == genus] <- genus_to_color[genus]
  data$tree_color_label[data$Genus == genus] <- genus
  data$tree_color_MRCA[data$Genus == genus] <- mrca_label

  # Highlight this clade
  p <- p + geom_hilight(node = mrca_node, fill = genus_to_color[genus], alpha=.6,
                        type = "gradient", gradient.direction = 'rt',)
}

# Create a new column that combines the genome_id and Organism fields
data$tree_label <- paste(data$genome_id, data$Organism_short, sep=" - ")

# Write the data to a new CSV file
write.table(data, file = "assets/iTOL_annotation/tree_annotation.csv", sep = ",", row.names = FALSE)

p <- p %<+% data + geom_tippoint(aes(color=Genus), size=3, show.legend = TRUE) + 
                   geom_tiplab(aes(label=tree_label, offset = 0.5)) + hexpand(.4)

# Set the color scale manually
p <- p + scale_color_manual(values = genus_to_color)

# Move the legend to the bottom
p <- p + theme(legend.position = 'bottom')

# Combine branch support and label
p <- p + geom_text(aes(label=ifelse(isTip, "", paste0(label, " (", format(round(branch.length, 2), nsmall = 2), ")"))), vjust=-0.5, hjust=1.1, size=2.8)

# Add a scale bar
p <- p + geom_treescale(x=0, y=0, offset=0.1)

# Display the plot
p

In [ ]:
outfile = Path("assets/tables/automlst_tree_table.csv")
outfile.parent.mkdir(parents=True, exist_ok=True)
outfile.write_text(df.to_csv(index=False))

display(HTML(DT(df.loc[:, ["genome_id", "genus_original", "species_original", "strain", "phylum", "class", "order", "family", "genus", "species"]], scrollX=True)))

[Download Table](assets/tables/automlst_tree_table.csv){:target="_blank" .md-button}

## Interactive Visualization with iTOL
For an enhanced, interactive visualization experience, users are encouraged to download the tree file and the corresponding annotation table. These files can be uploaded to [iTOL (Interactive Tree Of Life)](https://itol.embl.de/), a web-based tool for the display, manipulation, and annotation of phylogenetic trees. Please check the [iTOL help page](https://itol.embl.de/help.cgi) for the upload guide and annotation format.



In [ ]:
df_annotation = pd.read_csv("assets/iTOL_annotation/tree_annotation.csv")

# create label annotation file for iTOL
outfile_label = Path("assets/iTOL_annotation/iTOL_tree_label.txt")
outfile_label.parent.mkdir(parents=True, exist_ok=True)

## Write the header to the file
with open(outfile_label, 'w') as f:
    f.write("LABELS\n")
    f.write("SEPARATOR TAB\n")
    f.write("DATA\n")

## Write the data to the file
df_annotation[['genome_id', 'tree_label']].to_csv(outfile_label, sep='\t', header=False, index=False, mode='a')

In [ ]:
# create tree color annotation file for iTOL
outfile_color = Path("assets/iTOL_annotation/iTOL_tree_color.txt")

with open(outfile_color, 'w') as f:
    f.write("TREE_COLORS\n")
    f.write("SEPARATOR TAB\n")
    f.write("DATA\n")

df_annotation["tree_color_type"] = "range"
color_columns = ["tree_color_MRCA", "tree_color_type", "tree_color", "tree_color_label"]
df_color = df_annotation[~df_annotation[color_columns].duplicated()][color_columns]
#df_color['tree_color_MRCA'] = 'I' + df_color['tree_color_MRCA'].astype(str)

## Write the data to the file
df_color[color_columns].to_csv(outfile_color, sep='\t', header=False, index=False, mode='a')

In [ ]:
button = f'<a href="../assets/iTOL_annotation/automlst_tree_with_ids.newick" download class="md-button">Download iTOL Tree</a> <a href="../{outfile_label}" download class="md-button">Download iTOL Label</a> <a href="../{outfile_color}" download class="md-button">Download iTOL Color</a>'
display(Markdown(button))

In [ ]:
def create_itol_multiple_barchart_annotation(df, dataset_label, output_file, width=50, sep="COMMA"):
    # Define the iTOL template
    itol_template = """DATASET_MULTIBAR
#In multi-value bar charts, each ID is associated to multiple numeric values, which are displayed as a stacked or aligned bar chart
#lines starting with a hash are comments and ignored during parsing
#select the separator which is used to delimit the data below (TAB,SPACE or COMMA).This separator must be used throughout this file (except in the SEPARATOR line, which uses space).

#=================================================================
#                    MANDATORY SETTINGS                           #
#=================================================================#
SEPARATOR {sep}
DATASET_LABEL,{dataset_label}
COLOR,#ff0000
#=================================================================#
#                    OPTIONAL SETTINGS                            #
#=================================================================#
WIDTH,1{width}
MARGIN,0
SHOW_INTERNAL,0
HEIGHT_FACTOR,1
BAR_SHIFT,0
ALIGN_FIELDS,0
FIELD_LABELS,{field_labels}
FIELD_COLORS,{field_colors}
DATASET_SCALE,10,20,30,40,50
#=================================================================#
#       Actual data follows after the "DATA" keyword              #
#=================================================================#
DATA
{data}
"""

    # Define the field labels and colors
    field_labels = ','.join(df.columns)
    
    # Get the color palette from seaborn
    palette = sns.color_palette("Set2", len(df.columns))

    # Convert the RGB values to hexadecimal
    field_colors = ','.join([mcolors.rgb2hex(color) for color in palette])

    # Format the DataFrame to match the iTOL format
    df_itol = df.copy()
    df_itol.index.name = 'ID'
    df_itol = df_itol.reset_index()
    df_itol = df_itol.astype(str)
    data = '\n'.join(df_itol.apply(lambda x: ','.join(x), axis=1))

    # Fill the iTOL template with the field labels, colors, and data
    itol_annotation = itol_template.format(field_labels=field_labels, field_colors=field_colors, 
                                           width=width, data=data, sep=sep, dataset_label=dataset_label)

    # Write the iTOL annotation to a file
    with open(output_file, 'w') as f:
        f.write(itol_annotation)

In [ ]:
def create_itol_heatmap_annotation(df, dataset_label, output_file, strip_width=25, sep="COMMA", color_min="#ff0000", color_max="#0000ff", color_mid="#ffff00"):
    # Define the iTOL template
    itol_template = """DATASET_HEATMAP
#In heatmaps, each ID is associated to multiple numeric values, which are displayed as a set of colored boxes defined by a color gradient
#lines starting with a hash are comments and ignored during parsing
#=================================================================#
#                    MANDATORY SETTINGS                           #
#=================================================================#
#select the separator which is used to delimit the data below (TAB,SPACE or COMMA).This separator must be used throughout this file (except in the SEPARATOR line, which uses space).
#SEPARATOR TAB
#SEPARATOR SPACE
SEPARATOR {sep}

#label is used in the legend table (can be changed later)
DATASET_LABEL,{dataset_label}

#dataset color (can be changed later)
COLOR,#ff0000

#define labels for each individual field column
FIELD_LABELS,{field_labels}

#=================================================================#
#                    OPTIONAL SETTINGS                            #
#=================================================================#


#Heatmaps can have an optional Newick formatted tree assigned. Its leaf IDs must exactly match the dataset FIELD_LABELS.
#The tree will be used to sort the dataset fields, and will be displayed above the dataset. It can have branch lengths defined.
#All newlines and spaces should be stripped from the tree, and COMMA cannot be used as the dataset separator if a FIELD_TREE is provided.
#FIELD_TREE (((f1:0.2,f5:0.5):1,(f2_longer_one:0.2,f3:0.3):1.2):0.5,(f4:0.1,f6:0.5):0.8):1.52;
#FIELD_TREE (((f1,f5),(f2_longer_one,f3)),(f4,f6));
#FIELD_TREE (:0.1,:0.2,(:0.3,:0.4):0.5):0.0;

#=================================================================#
#     all other optional settings can be set or changed later     #
#           in the web interface (under 'Datasets' tab)           #
#=================================================================#

#left margin, used to increase/decrease the spacing to the next dataset. Can be negative, causing datasets to overlap.
MARGIN,0

#width of the individual boxes
STRIP_WIDTH,{strip_width}

#always show internal values; if set, values associated to internal nodes will be displayed even if these nodes are not collapsed. It could cause overlapping in the dataset display.
SHOW_INTERNAL,0

#if a FIELD_TREE is present, it can be hidden by setting this option to 0
SHOW_TREE,1

#define the heatmap gradient colors. Values in the dataset will be mapped onto the corresponding color gradient.
COLOR_MIN,{color_min}
COLOR_MAX,{color_max}

#you can specify a gradient with three colors (e.g red to yellow to green) by setting 'USE_MID_COLOR' to 1, and specifying the midpoint color
USE_MID_COLOR,1
COLOR_MID,{color_mid}

#Internal tree nodes can be specified using IDs directly, or using the 'last common ancestor' method described in iTOL help pages
#=================================================================#
#       Actual data follows after the "DATA" keyword              #
#=================================================================#
DATA
{data}
"""

    # Define the field labels and colors
    field_labels = ','.join(df.columns)
    
    # Format the DataFrame to match the iTOL format
    df_itol = df.copy()
    df_itol.index.name = 'ID'
    df_itol = df_itol.reset_index()
    df_itol = df_itol.astype(str)
    data = '\n'.join(df_itol.apply(lambda x: ','.join(x), axis=1))

    # Fill the iTOL template with the field labels, colors, and data
    itol_annotation = itol_template.format(field_labels=field_labels, color_min=color_min, color_max=color_max, color_mid=color_mid,
                                           strip_width=strip_width, data=data, sep=sep, dataset_label=dataset_label)

    # Write the iTOL annotation to a file
    with open(output_file, 'w') as f:
        f.write(itol_annotation)

In [ ]:
def create_itol_binary_annotation(df, dataset_label, field_shapes, field_colors, output_file, strip_width=25, sep="COMMA", color="#ff0000", height_factor=0.3, symbol_spacing=0):
    # Define the iTOL template
    itol_template = """DATASET_BINARY
#lines starting with a hash are comments and ignored during parsing
#select the separator which is used to delimit the data below (TAB,SPACE or COMMA).This separator must be used throught this file (except in the SEPARATOR line, which uses space).

#SEPARATOR TAB
#SEPARATOR SPACE
SEPARATOR {sep}

#label is used in the legend table (can be changed later)
DATASET_LABEL,{dataset_label}

#dataset color (can be changed later)
COLOR,{color}

#Binary datasets can contain one or more values for each node. Each value will be represented by a symbol (defined in FIELD_SHAPES) with corresponding color and label (from FIELD_COLORS and FIELD_LABELS). Possible values (defined under DATA below) for each node are 1 (filled shapes), 0 (empty shapes) and -1 (completely ommited).

#define colors for each individual field column (if not defined all symbols will use the main dataset color, defined in COLOR)
#shapes for each field column; possible choices are
#1: rectangle 
#2: circle
#3: star
#4: right pointing triangle
#5: left pointing triangle
FIELD_LABELS,{field_labels}
FIELD_COLORS,{field_colors}
FIELD_SHAPES,{field_shapes}

#all other optional settings can be set or changed later in the web interface (under 'Datasets' tab)

#show internal values; if set, values associated to internal nodes will be displayed even if these nodes are not collapsed. It could cause overlapping in the dataset display.
SHOW_INTERNAL,1

#left margin, used to increase/decrease the spacing to the next dataset. Can be negative, causing datasets to overlap.
MARGIN,0

#symbol height factor; Default symbol height will be slightly less than the available space between leaves, but you can set a multiplication factor here to increase/decrease it (values from 0 to 1 will decrease it, values above 1 will increase it)
HEIGHT_FACTOR,{height_factor}

#increase/decrease the spacing between individual levels, when there is more than one binary level defined 
SYMBOL_SPACING,{symbol_spacing}

#Internal tree nodes can be specified using IDs directly, or using the 'last common ancestor' method described in iTOL help pages
#Actual data follows after the "DATA" keyword
DATA
{data}
"""

    # Define the field labels and colors
    field_labels = ','.join([f"GCF_{c}" for c in df.columns])
    
    # Format the DataFrame to match the iTOL format
    df_itol = df.copy()
    df_itol.index.name = 'ID'
    df_itol = df_itol.reset_index()
    df_itol = df_itol.astype(str)
    data = '\n'.join(df_itol.apply(lambda x: ','.join(x), axis=1))
    field_shapes = ",".join(field_shapes)
    field_colors = ",".join(field_colors)

    # Fill the iTOL template with the field labels, colors, and data
    itol_annotation = itol_template.format(field_labels=field_labels, color=color, field_shapes=field_shapes,
                                           field_colors=field_colors, strip_width=strip_width, data=data, 
                                           sep=sep, symbol_spacing=symbol_spacing, height_factor=height_factor, 
                                           dataset_label=dataset_label)

    # Write the iTOL annotation to a file
    with open(output_file, 'w') as f:
        f.write(itol_annotation)

In [ ]:
# Initialize an empty list to store button items
button_items = []

# Define the path to the antiSMASH summary file
antismash_summary_path = Path(f"../tables/df_antismash_{antismash_version}_summary.csv")

# Check if the antiSMASH summary file exists
if antismash_summary_path.exists():
    # Read the antiSMASH summary file into a DataFrame
    df_antismash = pd.read_csv(antismash_summary_path).set_index("genome_id")
    
    # Calculate the number of complete BGCs by subtracting the number of BGCs on the contig edge from the total BGC count
    df_antismash["complete_bgcs"] = df_antismash["bgcs_count"] - df_antismash["bgcs_on_contig_edge"]
    
    # Select the 'complete_bgcs' and 'bgcs_on_contig_edge' columns and convert them to integers
    df_antismash_completeness = df_antismash.loc[:, ["complete_bgcs", "bgcs_on_contig_edge"]].fillna(0).astype(int)
    
    # Define the output file path
    outfile = Path(f"assets/iTOL_annotation/iTOL_antismash_{antismash_version}_completeness.txt")
    
    # Create the output directory if it doesn't exist
    outfile.parent.mkdir(parents=True, exist_ok=True)
    
    # Create the iTOL annotation
    create_itol_multiple_barchart_annotation(df_antismash_completeness, "BGC completeness", outfile)
    
    # Create a download button for the iTOL annotation
    button_link = f'<a href="../{outfile}" download class="md-button">Download antiSMASH barchart</a>'
    
    # Add the download button to the list of button items
    button_items.append(button_link)

    # Define the directory where the BIG-SCAPE results are stored
    bigscape_dir = report_dir /f"bigscape/for_cytoscape_antismash_{antismash_version}/"

    # Check if the BIG-SCAPE directory exists
    if bigscape_dir.exists():
        # Get the paths to the BIG-SCAPE output files
        bgc_table = [i for i in bigscape_dir.glob("*_df_clusters_0.30.csv")][0]
        gcf_table = [i for i in bigscape_dir.glob("*_df_families_0.30.csv")][0]
        mibig_table = [i for i in bigscape_dir.glob("*_df_known_0.30.csv")][0]
        gcf_presence_table = [i for i in bigscape_dir.glob("*_df_family_presence_0.30.csv")][0]
        network_table = [i for i in bigscape_dir.glob("*_df_network_0.30.csv")][0]

        # Read the BIG-SCAPE output files into DataFrames
        df_bgcs = pd.read_csv(bgc_table, index_col=0)
        df_gcf_presence = pd.read_csv(gcf_presence_table, index_col=0)
        df_gcfs = pd.read_csv(gcf_table, index_col=0)
        df_mibig = pd.read_csv(mibig_table, index_col=0)
        df_network = pd.read_csv(network_table, index_col=0)

        # Initialize a new DataFrame for the genomes
        df_genomes = pd.DataFrame()
        
        # Add a 'Genome ID' column to the genomes DataFrame
        df_genomes["Genome ID"] = df_antismash.index
        
        # Set the 'Genome ID' column as the index of the genomes DataFrame
        df_genomes = df_genomes.set_index("Genome ID", drop=False)
        
        # Add a 'BGCs' column to the genomes DataFrame
        df_genomes['BGCs'] = df_antismash.loc[df_genomes.index, 'bgcs_count']

        # Get the list of unique BIG-SCAPE classes
        bigscape_class_list = df_bgcs.bigscape_class.unique()

        # Loop over the index of the genomes DataFrame
        for i in df_genomes.index:
            # Get the genome ID for the current row
            gid = df_genomes.loc[i, 'Genome ID']
            
            # Check if 'known_family' is in the 'fam_type_0.30' column of the BGCs DataFrame
            if "known_family" in df_bgcs['fam_type_0.30']:
                # Count the number of known families for the current genome and store it in the 'Known BGCs' column
                df_genomes.loc[gid, 'Known BGCs'] = df_bgcs[df_bgcs.genome_id == gid].value_counts('fam_type_0.30')['known_family']
            
            # Check if 'unknown_family' is in the 'fam_type_0.30' column of the BGCs DataFrame
            if "unknown_family" in df_bgcs['fam_type_0.30']:
                # Count the number of unknown families for the current genome and store it in the 'Unknown BGCs' column
                df_genomes.loc[gid, 'Unknown BGCs'] = df_bgcs[df_bgcs.genome_id == gid].value_counts('fam_type_0.30')['unknown_family']
            
            # Count the number of unique BGCs for the current genome and store it in the 'Unique BGCs' column
            df_genomes.loc[gid, 'Unique BGCs'] = df_gcf_presence.loc[:, [str(idx) for idx in df_gcfs[df_gcfs.clusters_in_fam==1].index]].sum(1)[gid]
            
            # Count the number of each BIG-SCAPE class for the current genome
            df_bigscape_class_counts = df_bgcs[df_bgcs.genome_id == gid].value_counts('bigscape_class')
            
            # Loop over the list of unique BIG-SCAPE classes
            for bigscape_class in bigscape_class_list:
                # Check if the current BIG-SCAPE class is in the index of the BIG-SCAPE class counts DataFrame
                if bigscape_class in df_bigscape_class_counts.index:
                    # Store the count of the current BIG-SCAPE class in the genomes DataFrame
                    df_genomes.loc[gid, bigscape_class] = df_bigscape_class_counts[bigscape_class]
        
        # create absence presence matrix
        df_presence = df_gcf_presence.replace(0, -1)
        for gcf_id in df_gcfs.index:
            fam_type = df_gcfs.loc[gcf_id, "fam_type"]
            if fam_type == "unknown_family":
                df_presence[str(gcf_id)] = df_presence[str(gcf_id)].replace(1, 0)
        color = "#ff0000"
        shape = 1
        field_colors = [color for i in df_presence.columns]
        field_shapes = [str(shape) for i in df_presence.columns]
        
        # write absence presence matrix
        outfile = Path(f"assets/iTOL_annotation/iTOL_BiG-SCAPE_presence_antismash_{antismash_version}.txt")
        outfile.parent.mkdir(parents=True, exist_ok=True)
        create_itol_binary_annotation(df_presence, "GCF presence", field_shapes, field_colors, outfile, sep="COMMA", color="#ff0000")
        
        # Create a download button for the iTOL annotation
        button_link = f'<a href="../{outfile}" download class="md-button">Download BiG-SCAPE GCF presence</a>'
        
        # Add the download button to the list of button items
        button_items.append(button_link)
        
        # Get the column names of the genomes DataFrame excluding 'Genome ID', 'Unique BGCs', and 'BGCs'
        column_names = df_genomes.drop(['Genome ID', 'Unique BGCs', 'BGCs'], axis=1).columns

        # Select the columns with the obtained names, fill NA values with 0, and convert the data to integers
        df_bigscape_class_summary = df_genomes.loc[:, column_names].fillna(0).astype(int)

        # Define the output file path
        outfile = Path(f"assets/iTOL_annotation/iTOL_BiG-SCAPE_class_antismash_{antismash_version}.txt")

        # Create the output directory if it doesn't exist
        outfile.parent.mkdir(parents=True, exist_ok=True)

        # Create the iTOL annotation
        create_itol_multiple_barchart_annotation(df_bigscape_class_summary, "BiG-SCAPE Class", outfile)

        # Create a download button for the iTOL annotation
        button_link = f'<a href="../{outfile}" download class="md-button">Download BiG-SCAPE barchart</a>'

        # Add the download button to the list of button items
        button_items.append(button_link)

display(Markdown(" ".join(button_items)))

## References
<font size="2">

- Letunic I and Bork P (2021) Nucleic Acids Res doi: [10.1093/nar/gkab301](https://doi.org/10.1093/nar/gkab301) Interactive Tree Of Life (iTOL) v5: an online tool for phylogenetic tree display and annotation
- **G Yu**, DK Smith, H Zhu, Y Guan, TTY Lam<sup>\*</sup>. ggtree: an
    R package for visualization and annotation of phylogenetic trees
    with their covariates and other associated data. ***Methods in
    Ecology and Evolution***. 2017, 8(1):28-36. doi:
    [10.1111/2041-210X.12628](https://doi.org/10.1111/2041-210X.12628)

{% for i in project().rule_used['automlst-wrapper']['references'] %}
- *{{ i }}*
{% endfor %}
</font>